In [2]:
import torch
import cv2
import numpy as np

onnx_path = "../../model/yolov5s.onnx"  # 输出文件的路径
img = cv2.imread("./world_cup.jpg")

# load onnx model
import onnx
import onnxruntime as ort

onnx_model = onnx.load(onnx_path)

# 打印输入和输出
print("Inputs: ", onnx_model.graph.input)
print("Outputs: ", onnx_model.graph.output)

onnx.checker.check_model(onnx_model)
# 打印模型的层
# print(onnx.helper.printable_graph(onnx_model.graph))

# 创建一个运行时会话
sess = ort.InferenceSession(onnx_path)

# 输入是一个形状为 (1, 3, 640, 640) 的张量
# 将图像缩放到 640x640
img = cv2.resize(img, (640, 640))
# 将 BGR 图像转换为 RGB 图像
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# 将图像数据类型转换为 float32，并将值范围从 [0, 255] 缩放到 [0.0, 1.0]
img = img.astype('float32') / 255.0
# 将 HWC 图像转换为 CHW 图像
img = img.transpose((2, 0, 1))
# 添加一个新的维度，将 CHW 图像转换为 NCHW 图像
img_tensor = torch.from_numpy(img)

img_array = np.expand_dims(img_tensor, axis=0)
input_name = sess.get_inputs()[0].name

# 运行模型
result = sess.run(None, {input_name: img_array})

# 打印结果
print(result)


Inputs:  [name: "input_image"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 640
      }
      dim {
        dim_value: 640
      }
    }
  }
}
]
Outputs:  [name: "output0"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 1
      }
      dim {
        dim_value: 25200
      }
      dim {
        dim_value: 85
      }
    }
  }
}
]


2024-04-03 17:20:47.534629004 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 3538006, index: 2, mask: {3, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.


[array([[[3.8995953e+00, 3.5312743e+00, 7.4998097e+00, ...,
         1.3518035e-03, 5.6645274e-04, 1.5065968e-03],
        [1.2532164e+01, 3.8660750e+00, 2.4975594e+01, ...,
         1.3847649e-03, 7.8001618e-04, 1.5635192e-03],
        [1.8204559e+01, 4.1037483e+00, 3.2745758e+01, ...,
         1.2811124e-03, 7.7837706e-04, 1.7215014e-03],
        ...,
        [5.6543365e+02, 6.0541687e+02, 1.4089561e+02, ...,
         2.2886992e-03, 9.6750259e-04, 1.1696219e-03],
        [5.8790448e+02, 6.0599176e+02, 1.1021368e+02, ...,
         2.3210347e-03, 1.0295510e-03, 1.2754202e-03],
        [6.1625568e+02, 6.1786035e+02, 1.1451530e+02, ...,
         2.2132695e-03, 1.1677444e-03, 1.3512969e-03]]], dtype=float32)]


In [ ]:
# use atc convert onnx to om
!atc --model=./model/yolov5s.onnx --framework=5 --output=./model/yolov5s --input_format=NCHW  --soc_version=Ascend310B4 

607.20s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


ATC start working now, please wait for a moment.
/usr/local/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected 

In [5]:
from ais_bench.infer.interface import InferSession

om_path = "../../model/yolo.om"  # 输出文件的路径
session = InferSession(0, om_path)

input = torch.unsqueeze(img_tensor, dim=0).numpy()
# execute inference, inputs is ndarray list and outputs is ndarray list
output = session.infer(input, mode='static')

print(output)

[INFO] load model ../../model/yolo.om success
[INFO] create model description success
[array([[[3.89843750e+00, 3.54687500e+00, 7.50390625e+00, ...,
         1.35707855e-03, 5.68389893e-04, 1.50680542e-03],
        [1.25390625e+01, 3.86718750e+00, 2.49375000e+01, ...,
         1.38854980e-03, 7.82012939e-04, 1.56688690e-03],
        [1.82031250e+01, 4.10937500e+00, 3.27187500e+01, ...,
         1.28459930e-03, 7.82012939e-04, 1.72615051e-03],
        ...,
        [5.65500000e+02, 6.05500000e+02, 1.41000000e+02, ...,
         2.28691101e-03, 9.69886780e-04, 1.17492676e-03],
        [5.88000000e+02, 6.06000000e+02, 1.10187500e+02, ...,
         2.32315063e-03, 1.02806091e-03, 1.27410889e-03],
        [6.16000000e+02, 6.18000000e+02, 1.14375000e+02, ...,
         2.21633911e-03, 1.17015839e-03, 1.35040283e-03]]], dtype=float32)]
